# Detect the bottom dots on an image and use it to construct a spatial mapping

## Read, open and display the image

In [ ]:
# Load modules
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200

# Read image
path = './../data/calibration/exampledots.tiff'

bitdepth = 16

# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)

# Show image
plt.figure()
plt.imshow(image*1,origin='lower',cmap='gray')
plt.show()

## Extract grid data from loaded image

## Couple image to a spatial grid